# 🍔 AlexNet MOD1 (BatchNorm) - Tim Member 2

Notebook ini untuk melatih **AlexNet Mod1** dengan Batch Normalization.

**Assigned to:** [NAMA MEMBER 2]

---

In [ ]:
# ============================================================
# SETUP - Jalankan cell ini terlebih dahulu!
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import sys

PROJECT_PATH = '/content/drive/MyDrive/AlexNet_iFood2019'
REPO_PATH = '/content/alexnet-ifood2019'

# Clone repo
if not os.path.exists(REPO_PATH):
    !git clone https://github.com/deftorch/alexnet-ifood2019.git {REPO_PATH}

os.chdir(REPO_PATH)
sys.path.insert(0, REPO_PATH)

# Install dependencies
!pip install -q torch torchvision pandas numpy pillow scikit-learn matplotlib seaborn tqdm wandb

# Create symlinks to Drive
!rm -rf data checkpoints evaluation_results analysis_results 2>/dev/null
!ln -s {PROJECT_PATH}/dataset data
!ln -s {PROJECT_PATH}/checkpoints checkpoints
!ln -s {PROJECT_PATH}/evaluation_results evaluation_results
!ln -s {PROJECT_PATH}/analysis_results analysis_results

import torch
print(f"\n✅ Setup Complete!")
print(f"PyTorch: {torch.__version__}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NOT AVAILABLE'}")

In [ ]:
# ============================================================
# TRAINING: AlexNet Mod1 (BatchNorm)
# ============================================================

MODEL_NAME = "alexnet_mod1"  # ⚠️ JANGAN DIUBAH!

# Konfigurasi training
NUM_EPOCHS = 50
BATCH_SIZE = 128
LR = 0.01

print(f"🚀 Training {MODEL_NAME}...")
print(f"   Epochs: {NUM_EPOCHS}, Batch: {BATCH_SIZE}, LR: {LR}")

!python src/train.py \
    --data_dir data \
    --model_name {MODEL_NAME} \
    --num_epochs {NUM_EPOCHS} \
    --batch_size {BATCH_SIZE} \
    --lr {LR} \
    --save_dir checkpoints

In [ ]:
# ============================================================
# EVALUATION
# ============================================================

MODEL_NAME = "alexnet_mod1"

print(f"📊 Evaluating {MODEL_NAME}...")

!python src/evaluate.py \
    --data_dir data \
    --model_path checkpoints/{MODEL_NAME}_best.pth \
    --model_name {MODEL_NAME} \
    --split val \
    --output_dir evaluation_results

print(f"\n✅ Evaluation selesai! Hasil tersimpan di: evaluation_results/{MODEL_NAME}_val_metrics.json")

In [ ]:
# ============================================================
# LIHAT HASIL
# ============================================================

import json
import matplotlib.pyplot as plt

MODEL_NAME = "alexnet_mod1"

# Load history
with open(f'checkpoints/{MODEL_NAME}_history.json') as f:
    history = json.load(f)

# Load metrics
with open(f'evaluation_results/{MODEL_NAME}_val_metrics.json') as f:
    metrics = json.load(f)

# Print summary
print("=" * 50)
print(f"📈 HASIL {MODEL_NAME.upper()}")
print("=" * 50)
print(f"Best Val Accuracy: {max(history['val_acc']):.4f}")
print(f"Best Epoch: {history['val_acc'].index(max(history['val_acc'])) + 1}")
print(f"Test Accuracy: {metrics['accuracy']:.4f}")
print(f"Top-5 Accuracy: {metrics['top5_accuracy']:.4f}")
print(f"Macro F1: {metrics['macro_f1']:.4f}")
print("=" * 50)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
epochs = range(1, len(history['train_loss']) + 1)

axes[0].plot(epochs, history['train_loss'], label='Train')
axes[0].plot(epochs, history['val_loss'], label='Val')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(epochs, history['train_acc'], label='Train')
axes[1].plot(epochs, history['val_acc'], label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle(f'{MODEL_NAME} Training Curves')
plt.tight_layout()
plt.savefig(f'analysis_results/{MODEL_NAME}_curves.png', dpi=150)
plt.show()